In [2]:
import os
import sys
import logging
import numpy as np
import pandas as pd
import urlparse
os.sys.path.append("/home/sparrow/sparrow/malicious_url_cluster")


from config import cfg
from logger import logger
import preprocess
import vectorize
import clustering
import regex_url
import urlnormalize
import regex_domain
from file_io import load_urls, dump_urls
import joblib as jl
import url_forest
reload(url_forest)

<module 'url_forest' from '/home/sparrow/sparrow/malicious_url_cluster/url_forest.pyc'>

In [105]:
df_domain_url = pd.read_csv("../../data/EXP_domain/df_res_url.csv")
df_domain_id  = pd.read_csv("../../data/EXP_domain/df_res_sha256.csv")
df_domain_fp = pd.concat([df_domain_url.loc[df_domain_url.sha256==_]  \
                          for _ in list(df_domain_id.loc[df_domain_id.positives == 0].sha256)])
df_domain_tp = pd.concat([df_domain_url.loc[df_domain_url.sha256==_]  \
                          for _ in list(df_domain_id.loc[df_domain_id.positives >0].sha256)])

In [106]:
df_url_url = pd.read_csv("../../data/EXP_url/df_res_url.csv")
df_url_id  = pd.read_csv("../../data/EXP_url/df_res_sha256.csv")
df_url_fp  = pd.concat([df_url_url.loc[df_url_url.sha256==_]  \
                          for _ in list(df_url_id.loc[df_url_id.positives == 0].sha256)])
df_url_tp  = pd.concat([df_url_url.loc[df_url_url.sha256==_]  \
                          for _ in list(df_url_id.loc[df_url_id.positives >0].sha256)])

In [62]:
print "url regex detection\t%d\turl detection\t%d" %(len(set(df_url_url.sha256)), len(set(df_url_url.url)))
print "url regex fp\t\t%d\turl fp\t\t%d" %(len(set(df_url_fp.sha256)), len(set(df_url_fp.url)))


url regex detection	69	url detection	1145
url regex fp		29	url fp		87


In [63]:
print "domain regex detection\t%d\tdomain detection\t%d" %(len(set(df_domain_url.sha256)), len(set(df_domain_url.url)))
print "domain regex fp\t\t%d\tdomain fp\t\t%d" %(len(set(df_domain_fp.sha256)), len(set(df_domain_fp.url)))

domain regex detection	81	domain detection	1539
domain regex fp		37	domain fp		37


In [128]:
print "regex total detection\t%d\turl total detection\t%d" %(len(set(df_domain_url.sha256) | set(df_url_url.sha256)),
                                                             len(set(df_domain_url.url)|set(df_url_url.url)))
print "regex total fp\t\t%d\turl total fp\t\t%d" %(len(set(df_domain_fp.sha256) | set(df_url_fp.sha256)),
                                                             len(set(df_domain_fp.url)|set(df_url_fp.url)))

regex total detection	146	url total detection	2178
regex total fp		66	url total fp		124


In [197]:
safe_urls = load_urls("../../data/sangfor/safe_0419_mod.csv", csv = True)
safe_url_domain, safe_url_path = preprocess.make_url_split(safe_urls)
print len(safe_url_domain)

mal_urls = load_urls("../../data/EXP_domain/malware.csv", csv = True)
mal_url_domain, mal_url_path = preprocess.make_url_split(mal_urls)
print len(mal_url_domain)

2018-04-24 17:01:16 file_io.py [line:52] DEBUG	URLs Count:	1455678


467409


2018-04-24 17:01:42 file_io.py [line:52] DEBUG	URLs Count:	994836


360770


In [198]:
safe_domain_forest = url_forest.build_domain_forest(safe_url_domain)
mal_domain_forest = url_forest.build_domain_forest(mal_url_domain)
jl.dump(safe_domain_forest, "../../data/url_forest/safe_domain_forest_mod.jl.z")
jl.dump(mal_domain_forest, "../../data/url_forest/mal_domain_forest.jl.z")

building tree index:0
building tree index:1
building tree index:2
building tree index:3
building tree index:4
building tree index:5
building tree index:6
building tree index:7
building tree index:8
building tree index:9
building tree index:0
building tree index:1
building tree index:2
building tree index:3
building tree index:4
building tree index:5
building tree index:6
building tree index:7


['../../data/url_forest/mal_domain_forest.jl.z']

In [204]:
domain_reputation_thresh = 0.1

In [200]:
df = url_forest.analyze_domain_batch(list(df_domain_url.url), safe_domain_forest, mal_domain_forest, verbose = False)
df_publish = df.loc[df.domain_reputation>domain_reputation_thresh]

sel = [i in set(df_publish.url) for i in df_domain_url.url]
df_domain_url_p = df_domain_url.loc[sel]
sel = [i in set(df_publish.url) for i in df_domain_fp.url]
df_domain_fp_p = df_domain_fp.loc[sel]

print "domain regex detection\t%d\tdomain detection\t%d" %(len(set(df_domain_url_p.sha256)), len(set(df_domain_url_p.url)))
print "domain regex fp\t\t%d\tdomain fp\t\t%d" %(len(set(df_domain_fp_p.sha256)), len(set(df_domain_fp_p.url)))

domain regex detection	29	domain detection	1485
domain regex fp		3	domain fp		3


In [205]:
df = url_forest.analyze_domain_batch(list(df_url_url.url), safe_domain_forest, mal_domain_forest, verbose = False)
df_publish = df.loc[df.domain_reputation>domain_reputation_thresh]

sel = [i in set(df_publish.url) for i in df_url_url.url]
df_url_url_p = df_url_url.loc[sel]
sel = [i in set(df_publish.url) for i in df_url_fp.url]
df_url_fp_p = df_url_fp.loc[sel]

print "url regex detection\t%d\turl detection\t%d" %(len(set(df_url_url_p.sha256)), len(set(df_url_url_p.url)))
print "url regex fp\t\t%d\turl fp\t\t%d" %(len(set(df_url_fp_p.sha256)), len(set(df_url_fp_p.url)))

url regex detection	52	url detection	1088
url regex fp		17	url fp		35


In [206]:
print "regex total detection\t%d\turl total detection\t%d" %(len(set(df_domain_url_p.sha256) | set(df_url_url_p.sha256)),
                                                             len(set(df_domain_url_p.url)|set(df_url_url_p.url)))
print "regex total fp\t\t%d\turl total fp\t\t%d" %(len(set(df_domain_fp_p.sha256) | set(df_url_fp_p.sha256)),
                                                             len(set(df_domain_fp_p.url)|set(df_url_fp_p.url)))

regex total detection	77	url total detection	2067
regex total fp		20	url total fp		38


In [225]:
"""domain reputation analyze"""
url_forest.analyze_domain(safe_domain_forest, mal_domain_forest, domain="sdf.x.com", verbose = True)

0 0


-1

In [217]:
1088-35

1053

In [228]:
pprint.pprint(list(df_url_fp_p.url))

['http://api.ttkyy.net/mdparse/ckplayer/style.swf',
 'http://asset.qingshu520.com/assets/api/gift/video/76_android.zip',
 'http://asset.qingshu520.com/assets/api/gift/62.zip',
 'http://asset.qingshu520.com/assets/api/gift/video/6_android.zip',
 'http://tencent.myaora.net/test/upload/open/ca/2018/25858/apk/20180413161630_730_5859.apk',
 'http://asset.qingshu520.com/assets/api/gift/74.zip',
 'http://asset.qingshu520.com/assets/api/gift/78.zip',
 'http://asset.qingshu520.com/assets/api/gift/video/75_android.zip',
 'http://tu.dxcnd.cn/ggjc/update/20180327/10693454a770239c97385ddf28d7c40b-1803279990.db?auth_ver=2&appkey=50c82132bb394901f151ad96&nonce=1523254631674&model=T7&signmd5=608291140&vendor=CUBE&locale=en_US&pkg=cn.opda.a.phonoalbumshoushou&tk=UX5tr5IpcPXpC3cGrlflYA%3D%3D&vn=9.10.0&h=1824&w=1200&v=3911&ntt=WIFI&ie=862823021013547&lc=rZkSaTZAYNuzw7gP&dpi=320&sdk=19&s=e7f19d7de6b41917d03756ba1d944826',
 'http://tu.dxcnd.cn/ggjc/update/20180327/10693454a770239c97385ddf28d7c40b-180327999